## Identity Normalization in Residual Networks

This notebook contains the compact (residual) neural network, datasets and empirical results for the aforementioned paper.

### Dataset(s)

The first dataset used for empirical benchmarking is the Fruits-360 dataset, which was formerly a Kaggle competition. It consists of images of fruit labeled by fruit type and the variety. 

    1. There are a total of 47 types of fruit (e.g., Apple, Orange, Pear, etc) and 81 varieties.
    2. On average, there are 656 images per variety.
    3. Each image is 128x128 RGB.
    


In [1]:
!gsutil cp gs://cloud-samples-data/air/fruits360/fruits360-combined.zip .
!ls
!unzip -qn fruits360-combined.zip



Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update

Copying gs://cloud-samples-data/air/fruits360/fruits360-combined.zip...
\ [1 files][230.9 MiB/230.9 MiB]                                                
Operation completed over 1 objects/230.9 MiB.                                    
 arch-1.png	     cascade.ipynb	     fruits360-combined.zip   Training
 arch-2.png	     CAS-CNN.ipynb	     Save
 cascade-gap.ipynb  'CAS-CNN overview.jpg'   train_base_model.jpg


## Setup

Install:
    1. Tensorflow (1.13) machine learning framework
    2. Tensorflow integration of Keras API for building and training models.
    3. Numpy for general image/matrix manipulation.
    4. OpenCV for reading and preprocessing of images.
    5. Scikit-Learn for splitting datasets

In [1]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras import Sequential, Model, Input, optimizers
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, BatchNormalization, ReLU
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.layers as layers
from sklearn.model_selection import train_test_split

import numpy as np
import cv2

import os

## Steps

*Benchmark Fruits-360*
    1.  Create the Coarse Dataset for Training, Evaluation and Test
    2.  Define training routines
    3.  Run Benchmarks
    
*Benchmark Intel Image Classification*
    1.  Create the Dataset for Training, Evaluation and Test
    2.  Run Benchmarks
    
*Benchmark Columbia University COIL-100*
    11. Create the Dataset for Training, Evaluation and Test
    12. Run Benchmarks
    

# Coarse Fruits-360 Dataset


## (Step 1) Make Coarse Category Dataset


### Define Function to Make Coarse Train/Test Dataset from Fruits-360 dataset

This makes the by Fruit Type dataset.

In [2]:
def Fruits(root):
    n_label = 0
    images = []
    labels = []
    classes = {}
    
    os.chdir(root)
    classes_ = os.scandir('./')
    for class_ in classes_:
        print(class_.name)
        os.chdir(class_.name)
        classes[class_.name] = n_label

        # Finer Level Subdirectories per Coarse Level
        subclasses = os.scandir('./')
        for subclass in subclasses:
            os.chdir(subclass.name)
            files = os.listdir('./')
            for file in files:
                image = cv2.imread(file)
                images.append(image)
                labels.append(n_label)
                
            os.chdir('../')

        os.chdir('../')
        n_label += 1
    os.chdir('../')
    images = np.asarray(images)
    # standardization of the pixel data
    mean = np.mean(images)
    std  = np.std(images)
    images = ((images - mean) / std).astype(np.float32)
    # convert to one-hot encoded labels
    labels = to_categorical(labels, n_label)
    print("Images", images.shape, "Labels", labels.shape, "Classes", classes, "Mean", mean, "Stddev", std)

    # Split the processed image dataset into training and test data
    x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.20, shuffle=True)
    return x_train, x_test, y_train, y_test, classes, mean, std

### Generate the preprocessed Coarse Dataset

In [3]:
!free -m
x_train, x_test, y_train, y_test, fruits_classes, mean, std = Fruits('Training')
!free -m

              total        used        free      shared  buff/cache   available
Mem:          64146        4674       50986          96        8485       58881
Swap:         65187           0       65187
Avocado
Rambutan
Pineapple
Kiwi
Cantaloupe
Apple
Tamarillo
Physalis
Plum
Pitahaya
Guava
Limes
Grapefruit
Peach
Pomegranate
Nectarine
Apricot
Banana
Cherry
Mulberry
Raspberry
Cactus Fruit
Grape
Mandarine
Granadilla
Carambula
Passion Fruit
Lychee
Quince
Maracuja
Strawberry
Tangelo
Huckleberry
Orange
Dates
Melon
Pepino
Clementine
Papaya
Mango
Tomato
Salak
Kaki
Pear
Cocos
Lemon
Kumquats
Images (51258, 100, 100, 3) Labels (51258, 47) Classes {'Avocado': 0, 'Rambutan': 1, 'Pineapple': 2, 'Kiwi': 3, 'Cantaloupe': 4, 'Apple': 5, 'Tamarillo': 6, 'Physalis': 7, 'Plum': 8, 'Pitahaya': 9, 'Guava': 10, 'Limes': 11, 'Grapefruit': 12, 'Peach': 13, 'Pomegranate': 14, 'Nectarine': 15, 'Apricot': 16, 'Banana': 17, 'Cherry': 18, 'Mulberry': 19, 'Raspberry': 20, 'Cactus Fruit': 21, 'Grape': 22, 'Mandarine

### Split Coarse Training Dataset (by Fruit) further into Train and Validation

Next split out 10% of train to use for validation during training.

    - Train: 80%
        - Train: 90%
        - Validation: 10%
    - Test : 20%

In [4]:
# Split out 10% of Train to use for Validation
pivot = int(len(x_train) * 0.9)
x_val = x_train[pivot:]
y_val = y_train[pivot:]
x_train = x_train[:pivot]
y_train = y_train[:pivot]

print("train", x_train.shape, y_train.shape)
print("val  ", x_val.shape, y_val.shape)
print("test ", x_test.shape, y_test.shape)
!free -m

train (36905, 100, 100, 3) (36905, 47)
val   (4101, 100, 100, 3) (4101, 47)
test  (10252, 100, 100, 3) (10252, 47)
              total        used        free      shared  buff/cache   available
Mem:          64146       12012       43197          88        8935       51551
Swap:         65187           0       65187


## (Step 2) Define Training Routines

Define the routines we will use for training.

### Make Feeder

Prepare the Feeder mechanism for training the neural networkm using ImageDataGenerator. 

Add image augmentation for:

    1. Horizontal Flip
    2. Verticial  Flip
    3. Random Rotation +/- 30 degrees

In [5]:
def Feeder():
    datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=30, 
                                 width_shift_range=0.15, height_shift_range=0.15, shear_range=0.2)
    return datagen

### Make Trainer

Prepare a training session:

    1. Epochs defaults to 10
    2. Batch size defaults to 32
    3. Train with validation data
    4. Final evaluation with test data (holdout set).

In [6]:
def Train(model, datagen, x_train, y_train, x_test, y_test, epochs=10, batch_size=32):
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size, shuffle=True),
                    steps_per_epoch=len(x_train) / batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))
    scores = model.evaluate(x_train, y_train, verbose=1)
    print("Train", scores)

## (Step 3) Build Base Model

See paper for detailed description

In [10]:
from keras import Input, Model
import keras.layers as layers
import keras.optimizers as optimizers


def ConvNetA(input_shape, nclasses):
    ''' Compact Neural Network with Batch Normalization (Model 2)'''
    def stem(inputs):
        ''' The stem convolutional group '''
        # Two 3x3 convolutional layers, representational equivalent to single 5x5,
        # which reduces computational complexity (trainable weights) by 75%
        x = layers.Conv2D(64, 3, strides=2, padding='same', activation='relu')(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(64, 3, strides=2, padding='same', activation='relu')(x)
        x = layers.BatchNormalization()(x)
        # Reduce the feature map sizes by 75%
        x = layers.MaxPooling2D(2, strides=2)(x)
        return x
    
    def conv_groups(x):
        ''' Residual Groups (ResNet34 style) '''
        # transition convolution for identity link, delay downsampling
        shortcut = layers.Conv2D(128, (1,1), strides=1, padding='same')(x)
        x = layers.BatchNormalization()(x)
        
        # residual block - double filters (Replace two 3,3 with 3,3 and 1,1)
        x = layers.Conv2D(128, (3,3), strides=1, padding='same', activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(128, (3,1), strides=1, padding='same', activation='relu')(x)
        x = layers.BatchNormalization()(x)
        
        # identity link
        x = layers.add([shortcut, x])
        x = layers.Dropout(0.50)(x)
        
        # transition convolution for identity link
        shortcut = layers.Conv2D(256, (1,1), strides=1, padding='same')(x)
        
        # residual block - double filters
        x = layers.Conv2D(256, (3,3), strides=1, padding='same', activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(256, (3,1), strides=1, padding='same', activation='relu')(x)
        x = layers.BatchNormalization()(x)
        
        # identity link
        x = layers.add([shortcut, x])
        
        # pooling for final downsampling in convolutional layers
        x = layers.MaxPooling2D(2, strides=2, name='encoder')(x)

        return x
    
    def bottleneck(x):
        ''' The bottleneck layer '''
        # Use fast form of pooling: single value per feature map, 
        # which reduces the size substantially more than a Flatten().
        x = layers.GlobalAveragePooling2D(name='bottleneck')(x)
        return x
    
    def classifier(x, nclasses):
        ''' The classifier layer '''
        x = layers.Dense(nclasses, activation='softmax')(x)
        return x
    
    inputs = Input(input_shape)
    
    x = stem(inputs)
    x = conv_groups(x)
    x = bottleneck(x)
    outputs = classifier(x, nclasses)
    
    return Model(inputs, outputs)


def ConvNetB(input_shape, nclasses):
    ''' Compact Neural Network with Identity Normalization (Model 3)'''
    def stem(inputs):
        ''' The stem convolutional group '''
        # Two 3x3 convolutional layers, representational equivalent to single 5x5,
        # which reduces computational complexity (trainable weights) by 75%
        x = layers.Conv2D(64, 3, strides=2, padding='same', activation='relu')(inputs)
        x = layers.Conv2D(64, 3, strides=2, padding='same', activation='relu')(x)
        # Reduce the feature map sizes by 75%
        x = layers.MaxPooling2D(2, strides=2)(x)
        return x
    
    def conv_groups(x):
        ''' Residual Groups (ResNet34 style) '''
        # transition convolution for identity link, delay downsampling
        shortcut = layers.Conv2D(128, (1,1), strides=1, padding='same')(x)
        
        # residual block - double filters (Replace two 3,3 with 3,3 and 1,1)
        x = layers.Conv2D(128, (3,3), strides=1, padding='same', activation='relu')(x)
        x = layers.Conv2D(128, (1,1), strides=1, padding='same', activation='relu')(x)
        
        # identity link
        x = layers.add([shortcut, x])
        x = layers.Dropout(0.50)(x)
        
        # transition convolution for identity link
        shortcut = layers.Conv2D(256, (1,1), strides=1, padding='same')(x)
        
        # residual block - double filters
        x = layers.Conv2D(256, (3,3), strides=1, padding='same', activation='relu')(x)
        x = layers.Conv2D(256, (1,1), strides=1, padding='same', activation='relu')(x)
        
        # identity link
        x = layers.add([shortcut, x])
        x = layers.BatchNormalization()(x)
        
        # pooling for final downsampling in convolutional layers
        x = layers.MaxPooling2D(2, strides=2, name='encoder')(x)

        return x
    
    def bottleneck(x):
        ''' The bottleneck layer '''
        # Use fast form of pooling: single value per feature map, 
        # which reduces the size substantially more than a Flatten().
        x = layers.GlobalAveragePooling2D(name='bottleneck')(x)
        return x
    
    def classifier(x, nclasses):
        ''' The classifier layer '''
        x = layers.Dense(nclasses, activation='softmax')(x)
        return x
    
    inputs = Input(input_shape)
    
    x = stem(inputs)
    x = conv_groups(x)
    x = bottleneck(x)
    outputs = classifier(x, nclasses)
    
    return Model(inputs, outputs)
    

def ConvNetC(input_shape, nclasses):
    ''' Compact Neural Network without Normalization (Model 1)'''
    def stem(inputs):
        ''' The stem convolutional group '''
        # Two 3x3 convolutional layers, representational equivalent to single 5x5,
        # which reduces computational complexity (trainable weights) by 75%
        x = layers.Conv2D(64, 3, strides=2, padding='same', activation='relu')(inputs)
        x = layers.Conv2D(64, 3, strides=2, padding='same', activation='relu')(x)
        # Reduce the feature map sizes by 75%
        x = layers.MaxPooling2D(2, strides=2)(x)
        return x
    
    def conv_groups(x):
        ''' Residual Groups (ResNet34 style) '''
        # transition convolution for identity link, delay downsampling
        shortcut = layers.Conv2D(128, (1,1), strides=1, padding='same')(x)
        
        # residual block - double filters (Replace two 3,3 with 3,3 and 1,1)
        x = layers.Conv2D(128, (3,3), strides=1, padding='same', activation='relu')(x)
        x = layers.Conv2D(128, (1,1), strides=1, padding='same', activation='relu')(x)
        
        # identity link
        x = layers.add([shortcut, x])
        x = layers.Dropout(0.50)(x)
        
        # transition convolution for identity link
        shortcut = layers.Conv2D(256, (1,1), strides=1, padding='same')(x)
        
        # residual block - double filters
        x = layers.Conv2D(256, (3,3), strides=1, padding='same', activation='relu')(x)
        x = layers.Conv2D(256, (1,1), strides=1, padding='same', activation='relu')(x)
        
        # identity link
        x = layers.add([shortcut, x])
        
        # pooling for final downsampling in convolutional layers
        x = layers.MaxPooling2D(2, strides=2, name='encoder')(x)

        return x
    
    def bottleneck(x):
        ''' The bottleneck layer '''
        # Use fast form of pooling: single value per feature map, 
        # which reduces the size substantially more than a Flatten().
        x = layers.GlobalAveragePooling2D(name='bottleneck')(x)
        return x
    
    def classifier(x, nclasses):
        ''' The classifier layer '''
        x = layers.Dense(nclasses, activation='softmax')(x)
        return x
    
    inputs = Input(input_shape)
    
    x = stem(inputs)
    x = conv_groups(x)
    x = bottleneck(x)
    outputs = classifier(x, nclasses)
    
    return Model(inputs, outputs)


# Remove this (obsolete)
def ConvNetD(input_shape, nclasses):
    def stem(inputs):
        ''' The stem convolutional group '''
        # Two 3x3 convolutional layers, representational equivalent to single 5x5,
        # which reduces computational complexity (trainable weights) by 75%
        x = layers.Conv2D(64, 3, strides=2, padding='same', activation='relu')(inputs)
        x = layers.Conv2D(64, 3, strides=2, padding='same', activation='relu')(x)
        # Reduce the feature map sizes by 75%
        x = layers.MaxPooling2D(2, strides=2)(x)
        return x
    
    def conv_groups(x):
        ''' Residual Groups (ResNet34 style) '''
        # transition convolution for identity link, delay downsampling
        shortcut = layers.Conv2D(128, (1,1), strides=1, padding='same')(x)
        
        # residual block - double filters (Replace two 3,3 with 3,3 and 1,1)
        x = layers.Conv2D(128, (3,3), strides=1, padding='same', activation='relu')(x)
        x = layers.Conv2D(128, (1,1), strides=1, padding='same', activation='relu')(x)
        
        # identity link
        x = layers.add([shortcut, x])
        x = layers.BatchNormalization()(x)
        
        x = layers.Dropout(0.50)(x)
        
        # transition convolution for identity link
        shortcut = layers.Conv2D(256, (1,1), strides=1, padding='same')(x)
        
        # residual block - double filters
        x = layers.Conv2D(256, (3,3), strides=1, padding='same', activation='relu')(x)
        x = layers.Conv2D(256, (1,1), strides=1, padding='same', activation='relu')(x)
        
        # identity link
        x = layers.add([shortcut, x])
        x = layers.BatchNormalization()(x)
        
        # pooling for final downsampling in convolutional layers
        x = layers.MaxPooling2D(2, strides=2, name='encoder')(x)

        return x
    
    def bottleneck(x):
        ''' The bottleneck layer '''
        # Use fast form of pooling: single value per feature map, 
        # which reduces the size substantially more than a Flatten().
        x = layers.GlobalAveragePooling2D(name='bottleneck')(x)
        return x
    
    def classifier(x, nclasses):
        ''' The classifier layer '''
        x = layers.Dense(nclasses, activation='softmax')(x)
        return x
    
    inputs = Input(input_shape)
    
    x = stem(inputs)
    x = conv_groups(x)
    x = bottleneck(x)
    outputs = classifier(x, nclasses)
    
    return Model(inputs, outputs)

## Benchmark Fruits-360 with Models 2 and 3

In [16]:
import keras.optimizers
for lr in [0.1, 0.01, 0.001, 0.0001]:
    optimizer = optimizers.Adam(lr=lr)
    
    for i in range(3):
        print("RESNET WITH LAYER BATCHNORM, lr =", lr, "ITER = ", i)
        model = ConvNetA((100, 100, 3), y_train.shape[1])
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        datagen = Feeder()
        Train(model, datagen, x_train, y_train, x_val, y_val, 20)

        scores = model.evaluate(x_test, y_test, verbose=1)
        print("Test", scores)
    
    for i in range(3):
        print("RESNET WITH IDENTITY BATCHNORM, lr =", lr, "ITER = ", i)
        model = ConvNetB((100, 100, 3), y_train.shape[1])
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        datagen = Feeder()
        Train(model, datagen, x_train, y_train, x_val, y_val, 20)

        scores = model.evaluate(x_test, y_test, verbose=1)
        print("Test", scores)

RESNET WITH LAYER BATCHNORM, lr = 0.1 ITER =  0
Epoch 1/20
1154/1153 [==============================] - 258s 224ms/step - loss: 14.0790 - acc: 0.1258 - val_loss: 14.1333 - val_acc: 0.1231
Epoch 2/20
1154/1153 [==============================] - 253s 219ms/step - loss: 14.0836 - acc: 0.1262 - val_loss: 14.1333 - val_acc: 0.1231
Epoch 3/20
1154/1153 [==============================] - 267s 231ms/step - loss: 14.0836 - acc: 0.1262 - val_loss: 14.1333 - val_acc: 0.1231
Epoch 4/20
1154/1153 [==============================] - 265s 230ms/step - loss: 14.0836 - acc: 0.1262 - val_loss: 14.1333 - val_acc: 0.1231
Epoch 5/20
1154/1153 [==============================] - 260s 226ms/step - loss: 14.0848 - acc: 0.1262 - val_loss: 14.1333 - val_acc: 0.1231
Epoch 6/20
1154/1153 [==============================] - 265s 230ms/step - loss: 14.0859 - acc: 0.1261 - val_loss: 14.1333 - val_acc: 0.1231
Epoch 7/20
1154/1153 [==============================] - 262s 227ms/step - loss: 14.0836 - acc: 0.1262 - val_loss

1154/1153 [==============================] - 133s 116ms/step - loss: 15.9147 - acc: 0.0126 - val_loss: 15.9255 - val_acc: 0.0119
Epoch 8/20
1154/1153 [==============================] - 133s 116ms/step - loss: 15.9147 - acc: 0.0126 - val_loss: 15.9255 - val_acc: 0.0119
Epoch 9/20
1154/1153 [==============================] - 133s 116ms/step - loss: 15.9147 - acc: 0.0126 - val_loss: 15.9255 - val_acc: 0.0119
Epoch 10/20
1154/1153 [==============================] - 133s 115ms/step - loss: 15.9147 - acc: 0.0126 - val_loss: 15.9255 - val_acc: 0.0119
Epoch 11/20
1154/1153 [==============================] - 134s 116ms/step - loss: 15.9147 - acc: 0.0126 - val_loss: 15.9255 - val_acc: 0.0119
Epoch 12/20
1154/1153 [==============================] - 133s 116ms/step - loss: 15.9147 - acc: 0.0126 - val_loss: 15.9255 - val_acc: 0.0119
Epoch 13/20
1154/1153 [==============================] - 134s 116ms/step - loss: 15.9147 - acc: 0.0126 - val_loss: 15.9255 - val_acc: 0.0119
Epoch 14/20
1154/1153 [====

Epoch 15/20
1154/1153 [==============================] - 134s 116ms/step - loss: 0.0914 - acc: 0.9644 - val_loss: 0.1105 - val_acc: 0.9629
Epoch 16/20
1154/1153 [==============================] - 134s 116ms/step - loss: 0.0892 - acc: 0.9662 - val_loss: 0.0589 - val_acc: 0.9766
Epoch 17/20
1154/1153 [==============================] - 135s 117ms/step - loss: 0.0792 - acc: 0.9700 - val_loss: 0.1015 - val_acc: 0.9661
Epoch 18/20
1154/1153 [==============================] - 134s 116ms/step - loss: 0.0883 - acc: 0.9685 - val_loss: 0.0511 - val_acc: 0.9783
Epoch 19/20
1154/1153 [==============================] - 134s 116ms/step - loss: 0.0761 - acc: 0.9704 - val_loss: 0.1067 - val_acc: 0.9627
Epoch 20/20
36905/36905 [==============================] - 35s 958us/step
Train [0.10576896193150885, 0.9687847175179515]
10252/10252 [==============================] - 10s 963us/step
Test [0.11148275362918719, 0.9667381974248928]
RESNET WITH IDENTITY BATCHNORM, lr = 0.01 ITER =  2
Epoch 1/20
1154/1153 [

1154/1153 [==============================] - 141s 122ms/step - loss: 0.8122 - acc: 0.7501 - val_loss: 0.4444 - val_acc: 0.8544
Epoch 2/20
1154/1153 [==============================] - 136s 118ms/step - loss: 0.2592 - acc: 0.9112 - val_loss: 0.2677 - val_acc: 0.9076
Epoch 3/20
1154/1153 [==============================] - 135s 117ms/step - loss: 0.1444 - acc: 0.9503 - val_loss: 0.1509 - val_acc: 0.9427
Epoch 4/20
1154/1153 [==============================] - 136s 118ms/step - loss: 0.1009 - acc: 0.9631 - val_loss: 0.1262 - val_acc: 0.9534
Epoch 5/20
1154/1153 [==============================] - 137s 118ms/step - loss: 0.0822 - acc: 0.9688 - val_loss: 0.0725 - val_acc: 0.9734
Epoch 6/20
1154/1153 [==============================] - 137s 119ms/step - loss: 0.0736 - acc: 0.9715 - val_loss: 0.1004 - val_acc: 0.9624
Epoch 7/20
1154/1153 [==============================] - 138s 119ms/step - loss: 0.0626 - acc: 0.9755 - val_loss: 0.0603 - val_acc: 0.9746
Epoch 8/20
1154/1153 [=======================

1154/1153 [==============================] - 138s 120ms/step - loss: 0.0499 - acc: 0.9800 - val_loss: 0.1125 - val_acc: 0.9600
Epoch 10/20
1154/1153 [==============================] - 139s 121ms/step - loss: 0.0460 - acc: 0.9811 - val_loss: 0.0952 - val_acc: 0.9595
Epoch 11/20
1154/1153 [==============================] - 140s 121ms/step - loss: 0.0444 - acc: 0.9813 - val_loss: 0.0308 - val_acc: 0.9881
Epoch 12/20
1154/1153 [==============================] - 139s 120ms/step - loss: 0.0414 - acc: 0.9821 - val_loss: 0.0619 - val_acc: 0.9778
Epoch 13/20
1154/1153 [==============================] - 138s 120ms/step - loss: 0.0408 - acc: 0.9827 - val_loss: 0.0437 - val_acc: 0.9817
Epoch 14/20
1154/1153 [==============================] - 137s 118ms/step - loss: 0.0366 - acc: 0.9834 - val_loss: 0.0403 - val_acc: 0.9837
Epoch 15/20
1154/1153 [==============================] - 138s 119ms/step - loss: 0.0390 - acc: 0.9825 - val_loss: 0.0779 - val_acc: 0.9729
Epoch 16/20
1154/1153 [================

## Benchmark Fruits-360 with Model 1

In [18]:
import keras.optimizers
for lr in [0.1, 0.01, 0.001, 0.0001]:
    optimizer = optimizers.Adam(lr=lr)
    
    for i in range(3):
        print("RESNET WITHOUT BATCHNORM, lr =", lr, "ITER = ", i)
        model = ConvNetC((100, 100, 3), y_train.shape[1])
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        datagen = Feeder()
        Train(model, datagen, x_train, y_train, x_val, y_val, 20)

        scores = model.evaluate(x_test, y_test, verbose=1)
        print("Test", scores)

RESNET WITHOUT BATCHNORM, lr = 0.1 ITER =  0
Epoch 1/20
1154/1153 [==============================] - 130s 113ms/step - loss: 14.0752 - acc: 0.1261 - val_loss: 14.1333 - val_acc: 0.1231
Epoch 2/20
1154/1153 [==============================] - 124s 107ms/step - loss: 14.0863 - acc: 0.1261 - val_loss: 14.1333 - val_acc: 0.1231
Epoch 3/20
1154/1153 [==============================] - 124s 107ms/step - loss: 14.0825 - acc: 0.1263 - val_loss: 14.1333 - val_acc: 0.1231
Epoch 4/20
1154/1153 [==============================] - 125s 108ms/step - loss: 14.0848 - acc: 0.1262 - val_loss: 14.1333 - val_acc: 0.1231
Epoch 5/20
1154/1153 [==============================] - 125s 108ms/step - loss: 14.0852 - acc: 0.1261 - val_loss: 14.1333 - val_acc: 0.1231
Epoch 6/20
1154/1153 [==============================] - 126s 109ms/step - loss: 14.0859 - acc: 0.1261 - val_loss: 14.1333 - val_acc: 0.1231
Epoch 7/20
1154/1153 [==============================] - 124s 108ms/step - loss: 14.0848 - acc: 0.1262 - val_loss: 1

1154/1153 [==============================] - 125s 108ms/step - loss: 14.0825 - acc: 0.1263 - val_loss: 14.1333 - val_acc: 0.1231
Epoch 9/20
1154/1153 [==============================] - 125s 108ms/step - loss: 14.0848 - acc: 0.1262 - val_loss: 14.1333 - val_acc: 0.1231
Epoch 10/20
1154/1153 [==============================] - 125s 109ms/step - loss: 14.0859 - acc: 0.1261 - val_loss: 14.1333 - val_acc: 0.1231
Epoch 11/20
1154/1153 [==============================] - 125s 108ms/step - loss: 14.0859 - acc: 0.1261 - val_loss: 14.1333 - val_acc: 0.1231
Epoch 12/20
1154/1153 [==============================] - 125s 109ms/step - loss: 14.0859 - acc: 0.1261 - val_loss: 14.1333 - val_acc: 0.1231
Epoch 13/20
1154/1153 [==============================] - 125s 108ms/step - loss: 14.0848 - acc: 0.1262 - val_loss: 14.1333 - val_acc: 0.1231
Epoch 14/20
1154/1153 [==============================] - 125s 108ms/step - loss: 14.0825 - acc: 0.1263 - val_loss: 14.1333 - val_acc: 0.1231
Epoch 15/20
1154/1153 [===

1154/1153 [==============================] - 126s 109ms/step - loss: 0.0934 - acc: 0.9653 - val_loss: 0.1198 - val_acc: 0.9554
Epoch 17/20
1154/1153 [==============================] - 126s 110ms/step - loss: 0.0789 - acc: 0.9704 - val_loss: 0.1240 - val_acc: 0.9622
Epoch 18/20
1154/1153 [==============================] - 126s 109ms/step - loss: 0.0804 - acc: 0.9709 - val_loss: 0.0668 - val_acc: 0.9712
Epoch 19/20
1154/1153 [==============================] - 125s 109ms/step - loss: 0.0757 - acc: 0.9719 - val_loss: 0.0907 - val_acc: 0.9663
Epoch 20/20
36905/36905 [==============================] - 36s 982us/step
Train [0.09114295683952336, 0.9656144153908685]
10252/10252 [==============================] - 10s 982us/step
Test [0.09284801713881016, 0.9646898166444029]
RESNET WITHOUT BATCHNORM, lr = 0.0001 ITER =  2
Epoch 1/20
1154/1153 [==============================] - 137s 119ms/step - loss: 0.9475 - acc: 0.7070 - val_loss: 0.5586 - val_acc: 0.8183
Epoch 2/20
1154/1153 [=================

## Benchmark Fruits-360 with modified Model 3

In [22]:
import keras.optimizers
for lr in [0.1, 0.01, 0.001, 0.0001]:
    optimizer = optimizers.Adam(lr=lr)
    
    for i in range(3):
        print("RESNET WITH IDENTITY BATCHNORM, lr =", lr, "ITER = ", i)
        model = ConvNetD((100, 100, 3), y_train.shape[1])
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        datagen = Feeder()
        Train(model, datagen, x_train, y_train, x_val, y_val, 20)

        scores = model.evaluate(x_test, y_test, verbose=1)
        print("Test", scores)

RESNET WITH IDENTITY BATCHNORM, lr = 0.1 ITER =  0
Epoch 1/20
1154/1153 [==============================] - 152s 132ms/step - loss: 2.2398 - acc: 0.5031 - val_loss: 5.2279 - val_acc: 0.4172
Epoch 2/20
1154/1153 [==============================] - 144s 124ms/step - loss: 5.8521 - acc: 0.5582 - val_loss: 11.5262 - val_acc: 0.2770
Epoch 3/20
1154/1153 [==============================] - 144s 125ms/step - loss: 7.0004 - acc: 0.5518 - val_loss: 5.9531 - val_acc: 0.6194
Epoch 4/20
1154/1153 [==============================] - 146s 126ms/step - loss: 6.7228 - acc: 0.5753 - val_loss: 6.4827 - val_acc: 0.5935
Epoch 5/20
1154/1153 [==============================] - 145s 126ms/step - loss: 6.7868 - acc: 0.5737 - val_loss: 8.7041 - val_acc: 0.4557
Epoch 6/20
1154/1153 [==============================] - 144s 125ms/step - loss: 6.8281 - acc: 0.5722 - val_loss: 8.4538 - val_acc: 0.4728
Epoch 7/20
1154/1153 [==============================] - 145s 125ms/step - loss: 7.1023 - acc: 0.5563 - val_loss: 6.9259 

1154/1153 [==============================] - 146s 126ms/step - loss: 0.1198 - acc: 0.9553 - val_loss: 0.1202 - val_acc: 0.9617
Epoch 9/20
1154/1153 [==============================] - 145s 126ms/step - loss: 0.1018 - acc: 0.9625 - val_loss: 0.1419 - val_acc: 0.9593
Epoch 10/20
1154/1153 [==============================] - 145s 126ms/step - loss: 0.1041 - acc: 0.9608 - val_loss: 0.1534 - val_acc: 0.9500
Epoch 11/20
1154/1153 [==============================] - 145s 126ms/step - loss: 0.0973 - acc: 0.9633 - val_loss: 0.0736 - val_acc: 0.9742
Epoch 12/20
1154/1153 [==============================] - 145s 126ms/step - loss: 0.0839 - acc: 0.9677 - val_loss: 0.1091 - val_acc: 0.9637
Epoch 13/20
1154/1153 [==============================] - 145s 126ms/step - loss: 0.0903 - acc: 0.9656 - val_loss: 0.1152 - val_acc: 0.9588
Epoch 14/20
1154/1153 [==============================] - 145s 126ms/step - loss: 0.0855 - acc: 0.9682 - val_loss: 0.0337 - val_acc: 0.9861
Epoch 15/20
1154/1153 [=================

1154/1153 [==============================] - 146s 127ms/step - loss: 0.0358 - acc: 0.9833 - val_loss: 0.0254 - val_acc: 0.9895
Epoch 17/20
1154/1153 [==============================] - 147s 128ms/step - loss: 0.0337 - acc: 0.9837 - val_loss: 0.0619 - val_acc: 0.9734
Epoch 18/20
1154/1153 [==============================] - 146s 127ms/step - loss: 0.0338 - acc: 0.9837 - val_loss: 0.0488 - val_acc: 0.9773
Epoch 19/20
1154/1153 [==============================] - 146s 127ms/step - loss: 0.0321 - acc: 0.9851 - val_loss: 0.0297 - val_acc: 0.9868
Epoch 20/20
36905/36905 [==============================] - 41s 1ms/step
Train [0.023357738256325294, 0.9863975071128573]
10252/10252 [==============================] - 11s 1ms/step
Test [0.02381701098835489, 0.9877097151775264]
RESNET WITH IDENTITY BATCHNORM, lr = 0.0001 ITER =  2
Epoch 1/20
1154/1153 [==============================] - 157s 136ms/step - loss: 0.7896 - acc: 0.8102 - val_loss: 0.3030 - val_acc: 0.9161
Epoch 2/20
1154/1153 [==============

In [23]:
!ls ../

emnist		     fruits360-Cascade		 mnist
flowers		     housing.zip		 mnist.py
flowers-keras.ipynb  intel-image-classification  Untitled1.ipynb
fruits360	     keras.ipynb		 Untitled.ipynb


## Dataset for Intel Image Classification

In [12]:
import cv2
import numpy as np
from keras.utils import to_categorical
import random

root_train = "../intel-image-classification/seg_train"
root_test  = "../intel-image-classification/seg_test"

def loadImages(root):
    data = []
    labels = []
    
    classes = {}
    nclass = 0
    subdirs = os.scandir(root)
    for subdir in subdirs:
        classes[subdir.name] = nclass
        images = []
        
        files = os.scandir(subdir.path)
        for file in files:
            image = cv2.imread(file.path)
            image = cv2.resize(image, (150, 150))
            images.append(image)
            
        data.append(np.asarray(images))
        labels.append(np.asarray([nclass for _ in range(len(images))]))
        nclass += 1
        
    data = np.concatenate(np.asarray(data))
    mean = np.mean(data)
    std  = np.std(data)
    data = (( data - mean ) / std).astype(np.float32)
    labels = to_categorical(np.concatenate(labels))
    return data, labels, classes
        
data, labels, classes = loadImages(root_train)
n = data.shape[0]
random.seed(101)
random.shuffle(data)
random.seed(101)
random.shuffle(labels)

pivot = int(n * .9)
x_train = data[0:pivot]
y_train = labels[0:pivot]
x_val   = data[pivot:]
y_val   = labels[pivot:]
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

data = None
labels = None

x_test, y_test, _= loadImages(root_test)


(12630, 150, 150, 3) (12630, 6)
(1404, 150, 150, 3) (1404, 6)


## Benchmarks for Intel Image Classification for Models 1, 2 and 3

In [58]:
import keras.optimizers
for lr in [0.1, 0.01, 0.001, 0.0001]:
    optimizer = optimizers.Adam(lr=lr)
    
    for i in range(3):
        print("RESNET WITHOUT BATCHNORM, lr =", lr, "ITER = ", i)
        model = ConvNetC((150, 150, 3), y_train.shape[1])
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        datagen = Feeder()
        Train(model, datagen, x_train, y_train, x_val, y_val, 20)

        scores = model.evaluate(x_test, y_test, verbose=1)
        print("Test", scores)
    
    for i in range(3):
        print("RESNET WITH LAYER BATCHNORM, lr =", lr, "ITER = ", i)
        model = ConvNetA((150, 150, 3), y_train.shape[1])
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        datagen = Feeder()
        Train(model, datagen, x_train, y_train, x_val, y_val, 20)

        scores = model.evaluate(x_test, y_test, verbose=1)
        print("Test", scores)
    
    for i in range(3):
        print("RESNET WITH IDENTITY BATCHNORM, lr =", lr, "ITER = ", i)
        model = ConvNetB((150, 150, 3), y_train.shape[1])
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        datagen = Feeder()
        Train(model, datagen, x_train, y_train, x_val, y_val, 20)

        scores = model.evaluate(x_test, y_test, verbose=1)
        print("Test", scores)

RESNET WITHOUT BATCHNORM, lr = 0.1 ITER =  0
Epoch 1/20
395/394 [==============================] - 113s 287ms/step - loss: 7.6407 - acc: 0.5238 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 2/20
395/394 [==============================] - 102s 257ms/step - loss: 7.6542 - acc: 0.5251 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 3/20
395/394 [==============================] - 107s 270ms/step - loss: 7.6537 - acc: 0.5252 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 4/20
395/394 [==============================] - 102s 257ms/step - loss: 7.6542 - acc: 0.5251 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 5/20
395/394 [==============================] - 101s 256ms/step - loss: 7.6531 - acc: 0.5252 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 6/20
395/394 [==============================] - 101s 256ms/step - loss: 7.6560 - acc: 0.5250 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 7/20
395/394 [==============================] - 102s 258ms/step - loss: 7.6537 - acc: 0.5252 - val_loss: 13.1103 - val_acc: 0.1

Epoch 17/20
395/394 [==============================] - 103s 260ms/step - loss: 7.6560 - acc: 0.5250 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 18/20
395/394 [==============================] - 103s 261ms/step - loss: 7.6548 - acc: 0.5251 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 19/20
395/394 [==============================] - 104s 263ms/step - loss: 7.6554 - acc: 0.5250 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 20/20
12630/12630 [==============================] - 27s 2ms/step
Train [7.6545001464143025, 0.5250989707188293]
3000/3000 [==============================] - 6s 2ms/step
Test [13.297428731282553, 0.175]
RESNET WITH LAYER BATCHNORM, lr = 0.1 ITER =  0
Epoch 1/20
395/394 [==============================] - 230s 582ms/step - loss: 8.9678 - acc: 0.4420 - val_loss: 12.9037 - val_acc: 0.1994
Epoch 2/20
395/394 [==============================] - 220s 556ms/step - loss: 9.0188 - acc: 0.4405 - val_loss: 12.9037 - val_acc: 0.1994
Epoch 3/20
395/394 [==============================] - 2

Epoch 11/20
395/394 [==============================] - 217s 549ms/step - loss: 13.5909 - acc: 0.1568 - val_loss: 13.8680 - val_acc: 0.1396
Epoch 12/20
395/394 [==============================] - 215s 544ms/step - loss: 13.5935 - acc: 0.1566 - val_loss: 13.8336 - val_acc: 0.1417
Epoch 13/20
395/394 [==============================] - 215s 544ms/step - loss: 13.5736 - acc: 0.1579 - val_loss: 13.8565 - val_acc: 0.1403
Epoch 14/20
395/394 [==============================] - 215s 545ms/step - loss: 13.6285 - acc: 0.1545 - val_loss: 13.8680 - val_acc: 0.1396
Epoch 15/20
395/394 [==============================] - 215s 546ms/step - loss: 13.5776 - acc: 0.1576 - val_loss: 13.8565 - val_acc: 0.1403
Epoch 16/20
395/394 [==============================] - 215s 545ms/step - loss: 13.6552 - acc: 0.1528 - val_loss: 13.8450 - val_acc: 0.1410
Epoch 17/20
395/394 [==============================] - 215s 544ms/step - loss: 13.6203 - acc: 0.1550 - val_loss: 13.8680 - val_acc: 0.1396
Epoch 18/20
395/394 [======

Epoch 20/20
12630/12630 [==============================] - 28s 2ms/step
Train [7.6545001464143025, 0.5250989707188293]
3000/3000 [==============================] - 7s 2ms/step
Test [13.297428731282553, 0.175]
RESNET WITHOUT BATCHNORM, lr = 0.01 ITER =  2
Epoch 1/20
395/394 [==============================] - 116s 293ms/step - loss: 7.6365 - acc: 0.5249 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 2/20
395/394 [==============================] - 102s 257ms/step - loss: 7.6548 - acc: 0.5251 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 3/20
395/394 [==============================] - 102s 258ms/step - loss: 7.6548 - acc: 0.5251 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 4/20
395/394 [==============================] - 102s 259ms/step - loss: 7.6548 - acc: 0.5251 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 5/20
395/394 [==============================] - 101s 257ms/step - loss: 7.6560 - acc: 0.5250 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 6/20
395/394 [==============================] - 102s 2

395/394 [==============================] - 115s 291ms/step - loss: 0.8495 - acc: 0.6734 - val_loss: 1.5281 - val_acc: 0.4323
Epoch 9/20
395/394 [==============================] - 115s 292ms/step - loss: 0.8341 - acc: 0.6800 - val_loss: 1.6177 - val_acc: 0.4074
Epoch 10/20
395/394 [==============================] - 115s 291ms/step - loss: 0.8184 - acc: 0.6842 - val_loss: 1.4598 - val_acc: 0.4252
Epoch 11/20
395/394 [==============================] - 115s 291ms/step - loss: 0.8057 - acc: 0.6894 - val_loss: 1.5124 - val_acc: 0.4145
Epoch 12/20
395/394 [==============================] - 115s 291ms/step - loss: 0.7871 - acc: 0.6991 - val_loss: 1.3254 - val_acc: 0.4822
Epoch 13/20
395/394 [==============================] - 115s 291ms/step - loss: 0.7776 - acc: 0.7033 - val_loss: 1.6119 - val_acc: 0.4003
Epoch 14/20
395/394 [==============================] - 115s 292ms/step - loss: 0.7635 - acc: 0.7096 - val_loss: 1.3801 - val_acc: 0.5199
Epoch 15/20
395/394 [==============================] -

395/394 [==============================] - 217s 550ms/step - loss: 0.4436 - acc: 0.8309 - val_loss: 0.6599 - val_acc: 0.7821
Epoch 19/20
395/394 [==============================] - 219s 554ms/step - loss: 0.4355 - acc: 0.8360 - val_loss: 0.6806 - val_acc: 0.7628
Epoch 20/20
12630/12630 [==============================] - 51s 4ms/step
Train [0.3992283265928569, 0.8557403008615037]
3000/3000 [==============================] - 12s 4ms/step
Test [0.7266968308289846, 0.7390000001589457]
RESNET WITH LAYER BATCHNORM, lr = 0.001 ITER =  1
Epoch 1/20
395/394 [==============================] - 249s 629ms/step - loss: 1.0286 - acc: 0.6399 - val_loss: 1.3295 - val_acc: 0.4879
Epoch 2/20
395/394 [==============================] - 222s 563ms/step - loss: 0.8042 - acc: 0.6905 - val_loss: 1.1832 - val_acc: 0.5691
Epoch 3/20
395/394 [==============================] - 217s 550ms/step - loss: 0.7411 - acc: 0.7150 - val_loss: 1.0839 - val_acc: 0.5905
Epoch 4/20
395/394 [==============================] - 217

395/394 [==============================] - 104s 263ms/step - loss: 0.7386 - acc: 0.7116 - val_loss: 1.5157 - val_acc: 0.4879
Epoch 7/20
395/394 [==============================] - 103s 262ms/step - loss: 0.7296 - acc: 0.7154 - val_loss: 1.5333 - val_acc: 0.4459
Epoch 8/20
395/394 [==============================] - 104s 262ms/step - loss: 0.7100 - acc: 0.7230 - val_loss: 1.4136 - val_acc: 0.4808
Epoch 9/20
395/394 [==============================] - 104s 263ms/step - loss: 0.7111 - acc: 0.7299 - val_loss: 1.3817 - val_acc: 0.4822
Epoch 10/20
395/394 [==============================] - 103s 262ms/step - loss: 0.6864 - acc: 0.7351 - val_loss: 1.3002 - val_acc: 0.5128
Epoch 11/20
395/394 [==============================] - 103s 261ms/step - loss: 0.6823 - acc: 0.7384 - val_loss: 1.3112 - val_acc: 0.5477
Epoch 12/20
395/394 [==============================] - 103s 262ms/step - loss: 0.6713 - acc: 0.7392 - val_loss: 1.2400 - val_acc: 0.4964
Epoch 13/20
395/394 [==============================] - 1

Epoch 1/20
395/394 [==============================] - 259s 656ms/step - loss: 0.8759 - acc: 0.6763 - val_loss: 1.1489 - val_acc: 0.5520
Epoch 2/20
395/394 [==============================] - 224s 566ms/step - loss: 0.7005 - acc: 0.7323 - val_loss: 0.8484 - val_acc: 0.6788
Epoch 3/20
395/394 [==============================] - 223s 564ms/step - loss: 0.6416 - acc: 0.7593 - val_loss: 0.7948 - val_acc: 0.6966
Epoch 4/20
395/394 [==============================] - 231s 584ms/step - loss: 0.6142 - acc: 0.7703 - val_loss: 0.8289 - val_acc: 0.6923
Epoch 5/20
395/394 [==============================] - 226s 572ms/step - loss: 0.5771 - acc: 0.7771 - val_loss: 0.9196 - val_acc: 0.6425
Epoch 6/20
395/394 [==============================] - 225s 571ms/step - loss: 0.5634 - acc: 0.7885 - val_loss: 0.7324 - val_acc: 0.7187
Epoch 7/20
395/394 [==============================] - 219s 554ms/step - loss: 0.5401 - acc: 0.7947 - val_loss: 0.7435 - val_acc: 0.7179
Epoch 8/20
395/394 [============================

395/394 [==============================] - 120s 304ms/step - loss: 0.5260 - acc: 0.8031 - val_loss: 0.7756 - val_acc: 0.7187
Epoch 12/20
395/394 [==============================] - 120s 304ms/step - loss: 0.5237 - acc: 0.8070 - val_loss: 0.7059 - val_acc: 0.7372
Epoch 13/20
395/394 [==============================] - 120s 304ms/step - loss: 0.5126 - acc: 0.8060 - val_loss: 0.7515 - val_acc: 0.7130
Epoch 14/20
395/394 [==============================] - 120s 305ms/step - loss: 0.5009 - acc: 0.8134 - val_loss: 0.8458 - val_acc: 0.6702
Epoch 15/20
395/394 [==============================] - 120s 304ms/step - loss: 0.4949 - acc: 0.8139 - val_loss: 0.8784 - val_acc: 0.6752
Epoch 16/20
395/394 [==============================] - 121s 306ms/step - loss: 0.4884 - acc: 0.8174 - val_loss: 0.8304 - val_acc: 0.6994
Epoch 17/20
395/394 [==============================] - 120s 304ms/step - loss: 0.4813 - acc: 0.8193 - val_loss: 0.8522 - val_acc: 0.7201
Epoch 18/20
395/394 [==============================] 

## Benchmarks for Fruits-360 for He-normal for Models 1, 2 and 3

In [9]:
# He-Normal
import keras.optimizers
for lr in [0.1, 0.01, 0.001, 0.0001]:
    optimizer = optimizers.Adam(lr=lr)
    
    for i in range(3):
        print("RESNET WITHOUT BATCHNORM, lr =", lr, "ITER = ", i)
        model = ConvNetC((100, 100, 3), y_train.shape[1])
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        datagen = Feeder()
        Train(model, datagen, x_train, y_train, x_val, y_val, 20)

        scores = model.evaluate(x_test, y_test, verbose=1)
        print("Test", scores)
    
    for i in range(3):
        print("RESNET WITH LAYER BATCHNORM, lr =", lr, "ITER = ", i)
        model = ConvNetA((100, 100, 3), y_train.shape[1])
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        datagen = Feeder()
        Train(model, datagen, x_train, y_train, x_val, y_val, 20)

        scores = model.evaluate(x_test, y_test, verbose=1)
        print("Test", scores)
    
    for i in range(3):
        print("RESNET WITH IDENTITY BATCHNORM, lr =", lr, "ITER = ", i)
        model = ConvNetB((100, 100, 3), y_train.shape[1])
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        datagen = Feeder()
        Train(model, datagen, x_train, y_train, x_val, y_val, 20)

        scores = model.evaluate(x_test, y_test, verbose=1)
        print("Test", scores)

RESNET WITHOUT BATCHNORM, lr = 0.1 ITER =  0
Epoch 1/20
1154/1153 [==============================] - 122s 106ms/step - loss: 14.0882 - acc: 0.1258 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 2/20
1154/1153 [==============================] - 121s 105ms/step - loss: 14.0902 - acc: 0.1258 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 3/20
1154/1153 [==============================] - 120s 104ms/step - loss: 14.0869 - acc: 0.1260 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 4/20
1154/1153 [==============================] - 120s 104ms/step - loss: 14.0858 - acc: 0.1261 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 5/20
1154/1153 [==============================] - 121s 105ms/step - loss: 14.0891 - acc: 0.1259 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 6/20
1154/1153 [==============================] - 121s 105ms/step - loss: 14.0891 - acc: 0.1259 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 7/20
1154/1153 [==============================] - 121s 105ms/step - loss: 14.0902 - acc: 0.1258 - val_loss: 1

Epoch 8/20
1154/1153 [==============================] - 254s 220ms/step - loss: 15.9127 - acc: 0.0127 - val_loss: 15.9216 - val_acc: 0.0122
Epoch 9/20
1154/1153 [==============================] - 253s 219ms/step - loss: 15.9127 - acc: 0.0127 - val_loss: 15.9216 - val_acc: 0.0122
Epoch 10/20
1154/1153 [==============================] - 253s 219ms/step - loss: 15.9138 - acc: 0.0127 - val_loss: 15.9216 - val_acc: 0.0122
Epoch 11/20
1154/1153 [==============================] - 253s 219ms/step - loss: 15.9138 - acc: 0.0127 - val_loss: 15.9216 - val_acc: 0.0122
Epoch 12/20
1154/1153 [==============================] - 253s 219ms/step - loss: 15.9138 - acc: 0.0127 - val_loss: 15.9216 - val_acc: 0.0122
Epoch 13/20
1154/1153 [==============================] - 253s 219ms/step - loss: 15.9138 - acc: 0.0127 - val_loss: 15.9216 - val_acc: 0.0122
Epoch 14/20
1154/1153 [==============================] - 253s 219ms/step - loss: 15.9138 - acc: 0.0127 - val_loss: 15.9216 - val_acc: 0.0122
Epoch 15/20
115

Epoch 1/20
1154/1153 [==============================] - 135s 117ms/step - loss: 15.8999 - acc: 0.0126 - val_loss: 15.9609 - val_acc: 0.0098
Epoch 2/20
1154/1153 [==============================] - 133s 115ms/step - loss: 15.9155 - acc: 0.0126 - val_loss: 15.9570 - val_acc: 0.0100
Epoch 3/20
1154/1153 [==============================] - 133s 115ms/step - loss: 15.1679 - acc: 0.0589 - val_loss: 14.4910 - val_acc: 0.1010
Epoch 4/20
1154/1153 [==============================] - 133s 115ms/step - loss: 14.5007 - acc: 0.1003 - val_loss: 14.4870 - val_acc: 0.1012
Epoch 5/20
1154/1153 [==============================] - 133s 115ms/step - loss: 14.4962 - acc: 0.1006 - val_loss: 14.4752 - val_acc: 0.1019
Epoch 6/20
1154/1153 [==============================] - 133s 115ms/step - loss: 14.4887 - acc: 0.1011 - val_loss: 14.4792 - val_acc: 0.1017
Epoch 7/20
1154/1153 [==============================] - 133s 115ms/step - loss: 14.4643 - acc: 0.1026 - val_loss: 14.4438 - val_acc: 0.1039
Epoch 8/20
1154/1153

Epoch 8/20
1154/1153 [==============================] - 250s 217ms/step - loss: 14.0880 - acc: 0.1260 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 9/20
1154/1153 [==============================] - 249s 216ms/step - loss: 14.0880 - acc: 0.1260 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 10/20
1154/1153 [==============================] - 250s 217ms/step - loss: 14.0880 - acc: 0.1260 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 11/20
1154/1153 [==============================] - 249s 216ms/step - loss: 14.0891 - acc: 0.1259 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 12/20
1154/1153 [==============================] - 251s 217ms/step - loss: 14.0891 - acc: 0.1259 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 13/20
1154/1153 [==============================] - 249s 216ms/step - loss: 14.0891 - acc: 0.1259 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 14/20
1154/1153 [==============================] - 249s 216ms/step - loss: 14.0902 - acc: 0.1258 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 15/20
115

Epoch 1/20
1154/1153 [==============================] - 136s 118ms/step - loss: 2.3493 - acc: 0.3352 - val_loss: 2.1540 - val_acc: 0.3428
Epoch 2/20
1154/1153 [==============================] - 133s 115ms/step - loss: 0.9881 - acc: 0.6806 - val_loss: 0.9791 - val_acc: 0.6506
Epoch 3/20
1154/1153 [==============================] - 133s 115ms/step - loss: 0.5087 - acc: 0.8273 - val_loss: 0.7333 - val_acc: 0.7642
Epoch 4/20
1154/1153 [==============================] - 132s 115ms/step - loss: 0.3050 - acc: 0.8946 - val_loss: 1.8633 - val_acc: 0.6116
Epoch 5/20
1154/1153 [==============================] - 133s 115ms/step - loss: 0.2272 - acc: 0.9194 - val_loss: 0.8002 - val_acc: 0.7810
Epoch 6/20
1154/1153 [==============================] - 133s 116ms/step - loss: 0.1846 - acc: 0.9326 - val_loss: 0.1916 - val_acc: 0.9320
Epoch 7/20
1154/1153 [==============================] - 133s 115ms/step - loss: 0.1625 - acc: 0.9406 - val_loss: 0.6188 - val_acc: 0.8386
Epoch 8/20
1154/1153 [============

Epoch 9/20
1154/1153 [==============================] - 252s 218ms/step - loss: 0.0552 - acc: 0.9769 - val_loss: 0.1292 - val_acc: 0.9559
Epoch 10/20
1154/1153 [==============================] - 254s 220ms/step - loss: 0.0479 - acc: 0.9791 - val_loss: 0.1308 - val_acc: 0.9578
Epoch 11/20
1154/1153 [==============================] - 252s 218ms/step - loss: 0.0508 - acc: 0.9783 - val_loss: 0.0920 - val_acc: 0.9605
Epoch 12/20
1154/1153 [==============================] - 248s 215ms/step - loss: 0.0500 - acc: 0.9782 - val_loss: 0.2368 - val_acc: 0.9295
Epoch 13/20
1154/1153 [==============================] - 248s 215ms/step - loss: 0.0495 - acc: 0.9784 - val_loss: 0.2213 - val_acc: 0.9507
Epoch 14/20
1154/1153 [==============================] - 256s 222ms/step - loss: 0.0435 - acc: 0.9800 - val_loss: 0.0453 - val_acc: 0.9805
Epoch 15/20
1154/1153 [==============================] - 253s 219ms/step - loss: 0.0405 - acc: 0.9810 - val_loss: 0.0548 - val_acc: 0.9768
Epoch 16/20
1154/1153 [=====

1154/1153 [==============================] - 136s 118ms/step - loss: 0.0411 - acc: 0.9820 - val_loss: 0.0530 - val_acc: 0.9771
Epoch 18/20
1154/1153 [==============================] - 135s 117ms/step - loss: 0.0372 - acc: 0.9828 - val_loss: 0.0482 - val_acc: 0.9837
Epoch 19/20
1154/1153 [==============================] - 135s 117ms/step - loss: 0.0386 - acc: 0.9819 - val_loss: 0.0249 - val_acc: 0.9883
Epoch 20/20
36905/36905 [==============================] - 36s 976us/step
Train [0.025248373208029726, 0.986803956103509]
10252/10252 [==============================] - 10s 984us/step
Test [0.02779397345039938, 0.9865392118611003]
RESNET WITHOUT BATCHNORM, lr = 0.0001 ITER =  0
Epoch 1/20
1154/1153 [==============================] - 128s 111ms/step - loss: 1.4890 - acc: 0.5784 - val_loss: 0.9808 - val_acc: 0.7042
Epoch 2/20
1154/1153 [==============================] - 124s 107ms/step - loss: 0.5421 - acc: 0.8278 - val_loss: 0.5849 - val_acc: 0.7983
Epoch 3/20
1154/1153 [==================

1154/1153 [==============================] - 250s 217ms/step - loss: 0.0911 - acc: 0.9687 - val_loss: 0.0822 - val_acc: 0.9681
Epoch 4/20
1154/1153 [==============================] - 249s 215ms/step - loss: 0.0728 - acc: 0.9748 - val_loss: 0.1202 - val_acc: 0.9612
Epoch 5/20
1154/1153 [==============================] - 248s 215ms/step - loss: 0.0619 - acc: 0.9768 - val_loss: 0.0374 - val_acc: 0.9827
Epoch 6/20
1154/1153 [==============================] - 248s 215ms/step - loss: 0.0546 - acc: 0.9795 - val_loss: 0.0612 - val_acc: 0.9754
Epoch 7/20
1154/1153 [==============================] - 248s 215ms/step - loss: 0.0520 - acc: 0.9795 - val_loss: 0.0256 - val_acc: 0.9881
Epoch 8/20
1154/1153 [==============================] - 248s 215ms/step - loss: 0.0469 - acc: 0.9806 - val_loss: 0.0368 - val_acc: 0.9839
Epoch 9/20
1154/1153 [==============================] - 248s 215ms/step - loss: 0.0444 - acc: 0.9814 - val_loss: 0.0568 - val_acc: 0.9776
Epoch 10/20
1154/1153 [======================

## Benchmarks for Fruits-360 and batch=128 for Models 1, 2 and 3

In [11]:
# batch 128 / fruits-360
import keras.optimizers
for lr in [0.1, 0.01, 0.001, 0.0001]:
    optimizer = optimizers.Adam(lr=lr)
    
    for i in range(3):
        print("RESNET WITHOUT BATCHNORM, lr =", lr, "ITER = ", i)
        model = ConvNetC((100, 100, 3), y_train.shape[1])
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        datagen = Feeder()
        Train(model, datagen, x_train, y_train, x_val, y_val, epochs=20, batch_size=128)

        scores = model.evaluate(x_test, y_test, verbose=1)
        print("Test", scores)
    
    for i in range(3):
        print("RESNET WITH LAYER BATCHNORM, lr =", lr, "ITER = ", i)
        model = ConvNetA((100, 100, 3), y_train.shape[1])
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        datagen = Feeder()
        Train(model, datagen, x_train, y_train, x_val, y_val, epochs=20, batch_size=128)

        scores = model.evaluate(x_test, y_test, verbose=1)
        print("Test", scores)
    
    for i in range(3):
        print("RESNET WITH IDENTITY BATCHNORM, lr =", lr, "ITER = ", i)
        model = ConvNetB((100, 100, 3), y_train.shape[1])
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        datagen = Feeder()
        Train(model, datagen, x_train, y_train, x_val, y_val, epochs=20, batch_size=128)

        scores = model.evaluate(x_test, y_test, verbose=1)
        print("Test", scores)

RESNET WITHOUT BATCHNORM, lr = 0.1 ITER =  0
Epoch 1/20
289/288 [==============================] - 127s 438ms/step - loss: 14.0551 - acc: 0.1254 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 2/20
289/288 [==============================] - 118s 409ms/step - loss: 14.0900 - acc: 0.1258 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 3/20
289/288 [==============================] - 118s 410ms/step - loss: 14.0891 - acc: 0.1259 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 4/20
289/288 [==============================] - 120s 416ms/step - loss: 14.0900 - acc: 0.1258 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 5/20
289/288 [==============================] - 119s 412ms/step - loss: 14.0864 - acc: 0.1261 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 6/20
289/288 [==============================] - 119s 412ms/step - loss: 14.0873 - acc: 0.1260 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 7/20
289/288 [==============================] - 120s 415ms/step - loss: 14.0891 - acc: 0.1259 - val_loss: 14.1569 - val_a

289/288 [==============================] - 118s 409ms/step - loss: 14.0882 - acc: 0.1259 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 17/20
289/288 [==============================] - 119s 411ms/step - loss: 14.0891 - acc: 0.1259 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 18/20
289/288 [==============================] - 118s 410ms/step - loss: 14.0919 - acc: 0.1257 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 19/20
289/288 [==============================] - 119s 412ms/step - loss: 14.0882 - acc: 0.1259 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 20/20
36905/36905 [==============================] - 36s 964us/step
Train [14.088975397814679, 0.12589080070471348]
10252/10252 [==============================] - 10s 961us/step
Test [14.045948446796967, 0.12856028092079594]
RESNET WITH LAYER BATCHNORM, lr = 0.1 ITER =  0
Epoch 1/20
289/288 [==============================] - 264s 912ms/step - loss: 14.0715 - acc: 0.1251 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 2/20
289/288 [===================

289/288 [==============================] - 134s 463ms/step - loss: 15.9151 - acc: 0.0126 - val_loss: 15.9177 - val_acc: 0.0124
Epoch 3/20
289/288 [==============================] - 134s 464ms/step - loss: 15.9151 - acc: 0.0126 - val_loss: 15.9177 - val_acc: 0.0124
Epoch 4/20
289/288 [==============================] - 133s 462ms/step - loss: 15.9151 - acc: 0.0126 - val_loss: 15.9177 - val_acc: 0.0124
Epoch 5/20
289/288 [==============================] - 133s 461ms/step - loss: 15.9141 - acc: 0.0127 - val_loss: 15.9177 - val_acc: 0.0124
Epoch 6/20
289/288 [==============================] - 134s 464ms/step - loss: 15.9151 - acc: 0.0126 - val_loss: 15.9177 - val_acc: 0.0124
Epoch 7/20
289/288 [==============================] - 133s 461ms/step - loss: 15.9151 - acc: 0.0126 - val_loss: 15.9177 - val_acc: 0.0124
Epoch 8/20
289/288 [==============================] - 133s 462ms/step - loss: 15.9151 - acc: 0.0126 - val_loss: 15.9177 - val_acc: 0.0124
Epoch 9/20
289/288 [=========================

Epoch 18/20
289/288 [==============================] - 119s 411ms/step - loss: 14.0854 - acc: 0.1261 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 19/20
289/288 [==============================] - 119s 412ms/step - loss: 14.0864 - acc: 0.1261 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 20/20
36905/36905 [==============================] - 38s 1ms/step
Train [14.088975397814679, 0.12589080070471348]
10252/10252 [==============================] - 10s 1ms/step
Test [14.045948446796967, 0.12856028092079594]
RESNET WITHOUT BATCHNORM, lr = 0.01 ITER =  2
Epoch 1/20
289/288 [==============================] - 129s 446ms/step - loss: 14.0560 - acc: 0.1254 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 2/20
289/288 [==============================] - 119s 412ms/step - loss: 14.0845 - acc: 0.1262 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 3/20
289/288 [==============================] - 120s 417ms/step - loss: 14.0900 - acc: 0.1258 - val_loss: 14.1569 - val_acc: 0.1217
Epoch 4/20
289/288 [===============

Epoch 5/20
289/288 [==============================] - 135s 469ms/step - loss: 0.1594 - acc: 0.9412 - val_loss: 0.7071 - val_acc: 0.7932
Epoch 6/20
289/288 [==============================] - 136s 469ms/step - loss: 0.1213 - acc: 0.9545 - val_loss: 0.3262 - val_acc: 0.9083
Epoch 7/20
289/288 [==============================] - 136s 472ms/step - loss: 0.0972 - acc: 0.9625 - val_loss: 0.1742 - val_acc: 0.9337
Epoch 8/20
289/288 [==============================] - 136s 471ms/step - loss: 0.0918 - acc: 0.9642 - val_loss: 0.1632 - val_acc: 0.9473
Epoch 9/20
289/288 [==============================] - 136s 472ms/step - loss: 0.0789 - acc: 0.9689 - val_loss: 0.1679 - val_acc: 0.9573
Epoch 10/20
289/288 [==============================] - 136s 471ms/step - loss: 0.0753 - acc: 0.9688 - val_loss: 0.1988 - val_acc: 0.9359
Epoch 11/20
289/288 [==============================] - 140s 483ms/step - loss: 0.0587 - acc: 0.9755 - val_loss: 0.2343 - val_acc: 0.9154
Epoch 12/20
289/288 [=========================

289/288 [==============================] - 262s 906ms/step - loss: 0.2403 - acc: 0.9705 - val_loss: 0.5273 - val_acc: 0.9117
Epoch 16/20
289/288 [==============================] - 263s 908ms/step - loss: 0.2432 - acc: 0.9696 - val_loss: 0.3447 - val_acc: 0.9476
Epoch 17/20
289/288 [==============================] - 263s 910ms/step - loss: 0.2491 - acc: 0.9680 - val_loss: 0.3757 - val_acc: 0.9410
Epoch 18/20
289/288 [==============================] - 261s 902ms/step - loss: 0.2419 - acc: 0.9705 - val_loss: 0.3250 - val_acc: 0.9600
Epoch 19/20
289/288 [==============================] - 261s 902ms/step - loss: 0.2399 - acc: 0.9712 - val_loss: 0.3070 - val_acc: 0.9583
Epoch 20/20
36905/36905 [==============================] - 62s 2ms/step
Train [0.5028287088435132, 0.9087386532990109]
10252/10252 [==============================] - 17s 2ms/step
Test [0.489476295015831, 0.9056769410614106]
RESNET WITH LAYER BATCHNORM, lr = 0.001 ITER =  1
Epoch 1/20
289/288 [==============================] -

289/288 [==============================] - 122s 422ms/step - loss: 0.6621 - acc: 0.7912 - val_loss: 1.1527 - val_acc: 0.6742
Epoch 4/20
289/288 [==============================] - 122s 422ms/step - loss: 0.5334 - acc: 0.8255 - val_loss: 1.0636 - val_acc: 0.7223
Epoch 5/20
289/288 [==============================] - 122s 422ms/step - loss: 0.4495 - acc: 0.8518 - val_loss: 0.7962 - val_acc: 0.7674
Epoch 6/20
289/288 [==============================] - 122s 423ms/step - loss: 0.3963 - acc: 0.8663 - val_loss: 0.8706 - val_acc: 0.7564
Epoch 7/20
289/288 [==============================] - 122s 424ms/step - loss: 0.3477 - acc: 0.8832 - val_loss: 0.9464 - val_acc: 0.7825
Epoch 8/20
289/288 [==============================] - 122s 422ms/step - loss: 0.3225 - acc: 0.8886 - val_loss: 0.6644 - val_acc: 0.8093
Epoch 9/20
289/288 [==============================] - 122s 422ms/step - loss: 0.2912 - acc: 0.8990 - val_loss: 0.6034 - val_acc: 0.8244
Epoch 10/20
289/288 [==============================] - 122s

289/288 [==============================] - 264s 912ms/step - loss: 0.0357 - acc: 0.9836 - val_loss: 0.1084 - val_acc: 0.9678
Epoch 14/20
289/288 [==============================] - 265s 916ms/step - loss: 0.0330 - acc: 0.9846 - val_loss: 0.0231 - val_acc: 0.9890
Epoch 15/20
289/288 [==============================] - 264s 913ms/step - loss: 0.0333 - acc: 0.9838 - val_loss: 0.0301 - val_acc: 0.9861
Epoch 16/20
289/288 [==============================] - 264s 915ms/step - loss: 0.0322 - acc: 0.9839 - val_loss: 0.0427 - val_acc: 0.9812
Epoch 17/20
289/288 [==============================] - 264s 913ms/step - loss: 0.0314 - acc: 0.9841 - val_loss: 0.0232 - val_acc: 0.9883
Epoch 18/20
289/288 [==============================] - 264s 912ms/step - loss: 0.0329 - acc: 0.9845 - val_loss: 0.0702 - val_acc: 0.9744
Epoch 19/20
289/288 [==============================] - 264s 915ms/step - loss: 0.0297 - acc: 0.9849 - val_loss: 0.0567 - val_acc: 0.9790
Epoch 20/20
36905/36905 [============================

## Benchmarks for Intel Image Classification and batch=128 for Models 1, 2

In [13]:
# batch 128 / intel
import keras.optimizers
for lr in [0.1, 0.01, 0.001, 0.0001]:
    optimizer = optimizers.Adam(lr=lr)
    
    for i in range(3):
        print("RESNET WITHOUT BATCHNORM, lr =", lr, "ITER = ", i)
        model = ConvNetC((150, 150, 3), y_train.shape[1])
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        datagen = Feeder()
        Train(model, datagen, x_train, y_train, x_val, y_val, epochs=20, batch_size=128)

        scores = model.evaluate(x_test, y_test, verbose=1)
        print("Test", scores)
    
    for i in range(3):
        print("RESNET WITH LAYER BATCHNORM, lr =", lr, "ITER = ", i)
        model = ConvNetA((150, 150, 3), y_train.shape[1])
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        datagen = Feeder()
        Train(model, datagen, x_train, y_train, x_val, y_val, epochs=20, batch_size=128)

        scores = model.evaluate(x_test, y_test, verbose=1)
        print("Test", scores)
    
    for i in range(3):
        print("RESNET WITH IDENTITY BATCHNORM, lr =", lr, "ITER = ", i)
        model = ConvNetB((150, 150, 3), y_train.shape[1])
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        datagen = Feeder()
        Train(model, datagen, x_train, y_train, x_val, y_val, epochs=20, batch_size=128)

        scores = model.evaluate(x_test, y_test, verbose=1)
        print("Test", scores)

RESNET WITHOUT BATCHNORM, lr = 0.1 ITER =  0
Epoch 1/20
99/98 [==============================] - 124s 1s/step - loss: 7.5996 - acc: 0.5197 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 2/20
99/98 [==============================] - 105s 1s/step - loss: 7.6620 - acc: 0.5246 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 3/20
99/98 [==============================] - 105s 1s/step - loss: 7.6558 - acc: 0.5250 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 4/20
99/98 [==============================] - 105s 1s/step - loss: 7.6540 - acc: 0.5251 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 5/20
99/98 [==============================] - 105s 1s/step - loss: 7.6540 - acc: 0.5251 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 6/20
99/98 [==============================] - 103s 1s/step - loss: 7.6601 - acc: 0.5247 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 7/20
99/98 [==============================] - 103s 1s/step - loss: 7.6564 - acc: 0.5250 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 8/20
99/98 [=============

Epoch 19/20
99/98 [==============================] - 106s 1s/step - loss: 7.6546 - acc: 0.5251 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 20/20
12630/12630 [==============================] - 28s 2ms/step
Train [7.6545001464143025, 0.5250989707188293]
3000/3000 [==============================] - 7s 2ms/step
Test [13.297428731282553, 0.175]
RESNET WITH LAYER BATCHNORM, lr = 0.1 ITER =  0
Epoch 1/20
99/98 [==============================] - 245s 2s/step - loss: 7.6422 - acc: 0.5179 - val_loss: 13.0299 - val_acc: 0.1916
Epoch 2/20
99/98 [==============================] - 226s 2s/step - loss: 7.6591 - acc: 0.5248 - val_loss: 12.9840 - val_acc: 0.1944
Epoch 3/20
99/98 [==============================] - 227s 2s/step - loss: 7.6489 - acc: 0.5254 - val_loss: 12.9496 - val_acc: 0.1966
Epoch 4/20
99/98 [==============================] - 230s 2s/step - loss: 7.6496 - acc: 0.5254 - val_loss: 12.9496 - val_acc: 0.1966
Epoch 5/20
99/98 [==============================] - 227s 2s/step - loss: 7.6545 - 

99/98 [==============================] - 117s 1s/step - loss: 7.6546 - acc: 0.5251 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 11/20
99/98 [==============================] - 117s 1s/step - loss: 7.6540 - acc: 0.5251 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 12/20
99/98 [==============================] - 117s 1s/step - loss: 7.6534 - acc: 0.5252 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 13/20
99/98 [==============================] - 117s 1s/step - loss: 7.6546 - acc: 0.5251 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 14/20
99/98 [==============================] - 117s 1s/step - loss: 7.6620 - acc: 0.5246 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 15/20
99/98 [==============================] - 117s 1s/step - loss: 7.6558 - acc: 0.5250 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 16/20
99/98 [==============================] - 119s 1s/step - loss: 7.6534 - acc: 0.5252 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 17/20
99/98 [==============================] - 120s 1s/step - loss: 7.6515 

99/98 [==============================] - 224s 2s/step - loss: 12.1386 - acc: 0.2469 - val_loss: 12.0082 - val_acc: 0.2550
Epoch 3/20
99/98 [==============================] - 223s 2s/step - loss: 12.0820 - acc: 0.2504 - val_loss: 11.9508 - val_acc: 0.2585
Epoch 4/20
99/98 [==============================] - 223s 2s/step - loss: 12.0858 - acc: 0.2502 - val_loss: 11.8475 - val_acc: 0.2650
Epoch 5/20
99/98 [==============================] - 224s 2s/step - loss: 12.1825 - acc: 0.2442 - val_loss: 10.7798 - val_acc: 0.3312
Epoch 6/20
99/98 [==============================] - 224s 2s/step - loss: 12.4703 - acc: 0.2263 - val_loss: 12.8118 - val_acc: 0.2051
Epoch 7/20
99/98 [==============================] - 223s 2s/step - loss: 13.3467 - acc: 0.1719 - val_loss: 12.7200 - val_acc: 0.2108
Epoch 8/20
99/98 [==============================] - 223s 2s/step - loss: 13.3537 - acc: 0.1715 - val_loss: 12.6855 - val_acc: 0.2130
Epoch 9/20
99/98 [==============================] - 224s 2s/step - loss: 13.3912

12630/12630 [==============================] - 32s 3ms/step
Train [0.6752317362896337, 0.7327790973824541]
3000/3000 [==============================] - 8s 3ms/step
Test [1.505130288441976, 0.4126666666666667]
RESNET WITH IDENTITY BATCHNORM, lr = 0.01 ITER =  2
Epoch 1/20
99/98 [==============================] - 143s 1s/step - loss: 1.4547 - acc: 0.5077 - val_loss: 2.3463 - val_acc: 0.2229
Epoch 2/20
99/98 [==============================] - 117s 1s/step - loss: 1.0604 - acc: 0.5863 - val_loss: 1.5680 - val_acc: 0.2835
Epoch 3/20
99/98 [==============================] - 120s 1s/step - loss: 0.9743 - acc: 0.6083 - val_loss: 1.6445 - val_acc: 0.2991
Epoch 4/20
99/98 [==============================] - 123s 1s/step - loss: 0.9287 - acc: 0.6282 - val_loss: 1.6452 - val_acc: 0.2984
Epoch 5/20
99/98 [==============================] - 123s 1s/step - loss: 0.9164 - acc: 0.6387 - val_loss: 1.5867 - val_acc: 0.2863
Epoch 6/20
99/98 [==============================] - 123s 1s/step - loss: 0.8899 - ac

99/98 [==============================] - 224s 2s/step - loss: 0.5265 - acc: 0.8032 - val_loss: 0.8623 - val_acc: 0.7066
Epoch 13/20
99/98 [==============================] - 223s 2s/step - loss: 0.5072 - acc: 0.8107 - val_loss: 0.7796 - val_acc: 0.7130
Epoch 14/20
99/98 [==============================] - 223s 2s/step - loss: 0.4934 - acc: 0.8168 - val_loss: 0.7337 - val_acc: 0.7429
Epoch 15/20
99/98 [==============================] - 224s 2s/step - loss: 0.4760 - acc: 0.8225 - val_loss: 0.7224 - val_acc: 0.7429
Epoch 16/20
99/98 [==============================] - 224s 2s/step - loss: 0.4726 - acc: 0.8237 - val_loss: 0.7523 - val_acc: 0.7422
Epoch 17/20
99/98 [==============================] - 223s 2s/step - loss: 0.4733 - acc: 0.8244 - val_loss: 0.7686 - val_acc: 0.7358
Epoch 18/20
99/98 [==============================] - 224s 2s/step - loss: 0.4567 - acc: 0.8319 - val_loss: 0.6487 - val_acc: 0.7771
Epoch 19/20
99/98 [==============================] - 224s 2s/step - loss: 0.4519 - acc: 

99/98 [==============================] - 104s 1s/step - loss: 0.7788 - acc: 0.6961 - val_loss: 1.5639 - val_acc: 0.4872
Epoch 5/20
99/98 [==============================] - 105s 1s/step - loss: 0.7703 - acc: 0.6994 - val_loss: 1.4789 - val_acc: 0.4886
Epoch 6/20
99/98 [==============================] - 104s 1s/step - loss: 0.7450 - acc: 0.7082 - val_loss: 1.4195 - val_acc: 0.5071
Epoch 7/20
99/98 [==============================] - 104s 1s/step - loss: 0.7348 - acc: 0.7169 - val_loss: 1.4245 - val_acc: 0.4929
Epoch 8/20
99/98 [==============================] - 103s 1s/step - loss: 0.7227 - acc: 0.7229 - val_loss: 1.4359 - val_acc: 0.5299
Epoch 9/20
99/98 [==============================] - 104s 1s/step - loss: 0.7133 - acc: 0.7246 - val_loss: 1.4415 - val_acc: 0.5121
Epoch 10/20
99/98 [==============================] - 104s 1s/step - loss: 0.7036 - acc: 0.7258 - val_loss: 1.3493 - val_acc: 0.5321
Epoch 11/20
99/98 [==============================] - 104s 1s/step - loss: 0.6899 - acc: 0.729

Epoch 1/20
99/98 [==============================] - 257s 3s/step - loss: 0.9276 - acc: 0.6697 - val_loss: 1.4045 - val_acc: 0.5534
Epoch 2/20
99/98 [==============================] - 224s 2s/step - loss: 0.6969 - acc: 0.7330 - val_loss: 1.2625 - val_acc: 0.5670
Epoch 3/20
99/98 [==============================] - 224s 2s/step - loss: 0.6396 - acc: 0.7540 - val_loss: 1.2456 - val_acc: 0.5869
Epoch 4/20
99/98 [==============================] - 224s 2s/step - loss: 0.6082 - acc: 0.7654 - val_loss: 0.9752 - val_acc: 0.6410
Epoch 5/20
99/98 [==============================] - 225s 2s/step - loss: 0.5809 - acc: 0.7798 - val_loss: 0.9338 - val_acc: 0.6538
Epoch 6/20
99/98 [==============================] - 225s 2s/step - loss: 0.5613 - acc: 0.7933 - val_loss: 0.9617 - val_acc: 0.6581
Epoch 7/20
99/98 [==============================] - 225s 2s/step - loss: 0.5342 - acc: 0.8004 - val_loss: 0.7944 - val_acc: 0.7073
Epoch 8/20
99/98 [==============================] - 225s 2s/step - loss: 0.5291 - a

## Dataset for Columbia University COIL-100

In [30]:
import cv2
import numpy as np
from keras.utils import to_categorical
import random
from sklearn.model_selection import train_test_split

root = '/usr/local/google/home/aferlitsch/Desktop/Datasets/coil-100'
def loadImages(root):
    nfiles=0
    images = []
    labels = []
    
    classes = {}
    nclass = 0
    files = os.scandir(root)
    for file in files:
        pair = file.name.split('_')
        label = pair[0][3:]
        image = cv2.imread(file.path)
        if image is None:
            continue
        image = cv2.resize(image, (128, 128))
        images.append(image)
        labels.append(int(label))
        nfiles += 1
        
        
    return nfiles, np.asarray(images).astype(np.float32), np.asarray(labels)
        
nfiles, images, labels = loadImages(root)
print(nfiles)

labels = to_categorical(labels)
mean = np.mean(images)
std  = np.mean(images)
images = (images - mean) / std

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.20, random_state=42)

pivot = int(len(x_train) * 0.9)
x_val = x_train[pivot:]
y_val = y_train[pivot:]
x_train = x_train[:pivot]
y_train = y_train[:pivot]
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(x_val.shape, y_val.shape)

7200
(5184, 128, 128, 3) (5184, 101)
(1440, 128, 128, 3) (1440, 101)
(576, 128, 128, 3) (576, 101)


## Benchmarks for COIL-100 for Models 1, 2 and 3

In [31]:
# COIL-100
import keras.optimizers
for lr in [0.1, 0.01, 0.001, 0.0001]:
    optimizer = optimizers.Adam(lr=lr)
    
    for i in range(3):
        print("RESNET WITHOUT BATCHNORM, lr =", lr, "ITER = ", i)
        model = ConvNetC((128, 128, 3), y_train.shape[1])
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        datagen = Feeder()
        Train(model, datagen, x_train, y_train, x_val, y_val, epochs=20, batch_size=32)

        scores = model.evaluate(x_test, y_test, verbose=1)
        print("Test", scores)
    
    for i in range(3):
        print("RESNET WITH LAYER BATCHNORM, lr =", lr, "ITER = ", i)
        model = ConvNetA((128, 128, 3), y_train.shape[1])
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        datagen = Feeder()
        Train(model, datagen, x_train, y_train, x_val, y_val, epochs=20, batch_size=32)

        scores = model.evaluate(x_test, y_test, verbose=1)
        print("Test", scores)
    
    for i in range(3):
        print("RESNET WITH IDENTITY BATCHNORM, lr =", lr, "ITER = ", i)
        model = ConvNetB((128, 128, 3), y_train.shape[1])
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        datagen = Feeder()
        Train(model, datagen, x_train, y_train, x_val, y_val, epochs=20, batch_size=32)

        scores = model.evaluate(x_test, y_test, verbose=1)
        print("Test", scores)

RESNET WITHOUT BATCHNORM, lr = 0.1 ITER =  0
Epoch 1/20
162/162 [==============================] - 64s 397ms/step - loss: 15.8853 - acc: 0.0102 - val_loss: 15.9782 - val_acc: 0.0087
Epoch 2/20
162/162 [==============================] - 31s 193ms/step - loss: 15.9502 - acc: 0.0104 - val_loss: 15.9782 - val_acc: 0.0087
Epoch 3/20
162/162 [==============================] - 31s 191ms/step - loss: 15.9502 - acc: 0.0104 - val_loss: 15.9782 - val_acc: 0.0087
Epoch 4/20
162/162 [==============================] - 31s 189ms/step - loss: 15.9502 - acc: 0.0104 - val_loss: 15.9782 - val_acc: 0.0087
Epoch 5/20
162/162 [==============================] - 31s 190ms/step - loss: 15.9502 - acc: 0.0104 - val_loss: 15.9782 - val_acc: 0.0087
Epoch 6/20
162/162 [==============================] - 32s 195ms/step - loss: 15.9502 - acc: 0.0104 - val_loss: 15.9782 - val_acc: 0.0087
Epoch 7/20
162/162 [==============================] - 31s 191ms/step - loss: 15.9502 - acc: 0.0104 - val_loss: 15.9782 - val_acc: 0.0

Epoch 10/20
162/162 [==============================] - 63s 392ms/step - loss: 15.9782 - acc: 0.0087 - val_loss: 15.9222 - val_acc: 0.0122
Epoch 11/20
162/162 [==============================] - 64s 393ms/step - loss: 15.9782 - acc: 0.0087 - val_loss: 15.9222 - val_acc: 0.0122
Epoch 12/20
162/162 [==============================] - 64s 394ms/step - loss: 15.9782 - acc: 0.0087 - val_loss: 15.9222 - val_acc: 0.0122
Epoch 13/20
162/162 [==============================] - 64s 393ms/step - loss: 15.9782 - acc: 0.0087 - val_loss: 15.9222 - val_acc: 0.0122
Epoch 14/20
162/162 [==============================] - 63s 392ms/step - loss: 15.9782 - acc: 0.0087 - val_loss: 15.9222 - val_acc: 0.0122
Epoch 15/20
162/162 [==============================] - 63s 392ms/step - loss: 15.9782 - acc: 0.0087 - val_loss: 15.9222 - val_acc: 0.0122
Epoch 16/20
162/162 [==============================] - 64s 393ms/step - loss: 15.9782 - acc: 0.0087 - val_loss: 15.9222 - val_acc: 0.0122
Epoch 17/20
162/162 [=============

Epoch 4/20
162/162 [==============================] - 34s 212ms/step - loss: 15.7823 - acc: 0.0208 - val_loss: 15.8383 - val_acc: 0.0174
Epoch 5/20
162/162 [==============================] - 34s 212ms/step - loss: 15.7947 - acc: 0.0201 - val_loss: 15.8383 - val_acc: 0.0174
Epoch 6/20
162/162 [==============================] - 34s 211ms/step - loss: 15.7916 - acc: 0.0203 - val_loss: 15.8383 - val_acc: 0.0174
Epoch 7/20
162/162 [==============================] - 34s 212ms/step - loss: 15.7885 - acc: 0.0204 - val_loss: 15.8383 - val_acc: 0.0174
Epoch 8/20
162/162 [==============================] - 34s 212ms/step - loss: 15.7885 - acc: 0.0204 - val_loss: 15.8383 - val_acc: 0.0174
Epoch 9/20
162/162 [==============================] - 35s 214ms/step - loss: 15.7823 - acc: 0.0208 - val_loss: 15.8383 - val_acc: 0.0174
Epoch 10/20
162/162 [==============================] - 34s 213ms/step - loss: 15.7916 - acc: 0.0203 - val_loss: 15.8383 - val_acc: 0.0174
Epoch 11/20
162/162 [===================

162/162 [==============================] - 64s 394ms/step - loss: 15.9564 - acc: 0.0100 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 14/20
162/162 [==============================] - 64s 396ms/step - loss: 15.9564 - acc: 0.0100 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 15/20
162/162 [==============================] - 64s 393ms/step - loss: 15.9564 - acc: 0.0100 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 16/20
162/162 [==============================] - 64s 393ms/step - loss: 15.9564 - acc: 0.0100 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 17/20
162/162 [==============================] - 64s 393ms/step - loss: 15.9564 - acc: 0.0100 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 18/20
162/162 [==============================] - 64s 392ms/step - loss: 15.9564 - acc: 0.0100 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 19/20
162/162 [==============================] - 64s 393ms/step - loss: 15.9564 - acc: 0.0100 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 20/20
5184/5

Epoch 2/20
162/162 [==============================] - 31s 193ms/step - loss: 1.0031 - acc: 0.6796 - val_loss: 1.0497 - val_acc: 0.6858
Epoch 3/20
162/162 [==============================] - 31s 194ms/step - loss: 0.5857 - acc: 0.8183 - val_loss: 0.8771 - val_acc: 0.7170
Epoch 4/20
162/162 [==============================] - 31s 194ms/step - loss: 0.4063 - acc: 0.8661 - val_loss: 0.5529 - val_acc: 0.8403
Epoch 5/20
162/162 [==============================] - 31s 194ms/step - loss: 0.3008 - acc: 0.8976 - val_loss: 0.6478 - val_acc: 0.8003
Epoch 6/20
162/162 [==============================] - 31s 194ms/step - loss: 0.2235 - acc: 0.9223 - val_loss: 0.3530 - val_acc: 0.8854
Epoch 7/20
162/162 [==============================] - 32s 196ms/step - loss: 0.2257 - acc: 0.9238 - val_loss: 0.4712 - val_acc: 0.8507
Epoch 8/20
162/162 [==============================] - 32s 197ms/step - loss: 0.1694 - acc: 0.9431 - val_loss: 0.4507 - val_acc: 0.8750
Epoch 9/20
162/162 [==============================] - 3

Epoch 19/20
162/162 [==============================] - 64s 396ms/step - loss: 0.0543 - acc: 0.9836 - val_loss: 0.1408 - val_acc: 0.9566
Epoch 20/20
5184/5184 [==============================] - 16s 3ms/step
Train [0.058167747275064484, 0.9822530864197531]
1440/1440 [==============================] - 5s 3ms/step
Test [0.07240489174404906, 0.9763888888888889]
RESNET WITH LAYER BATCHNORM, lr = 0.001 ITER =  1
Epoch 1/20
162/162 [==============================] - 111s 686ms/step - loss: 2.1218 - acc: 0.4925 - val_loss: 3.0826 - val_acc: 0.3646
Epoch 2/20
162/162 [==============================] - 64s 395ms/step - loss: 0.5984 - acc: 0.8218 - val_loss: 0.9802 - val_acc: 0.6875
Epoch 3/20
162/162 [==============================] - 65s 400ms/step - loss: 0.3190 - acc: 0.9039 - val_loss: 1.5033 - val_acc: 0.6736
Epoch 4/20
162/162 [==============================] - 64s 395ms/step - loss: 0.2456 - acc: 0.9244 - val_loss: 0.4023 - val_acc: 0.8854
Epoch 5/20
162/162 [==============================

Epoch 8/20
162/162 [==============================] - 32s 196ms/step - loss: 0.7326 - acc: 0.7747 - val_loss: 1.0419 - val_acc: 0.6562
Epoch 9/20
162/162 [==============================] - 32s 195ms/step - loss: 0.6862 - acc: 0.7903 - val_loss: 0.9205 - val_acc: 0.7465
Epoch 10/20
162/162 [==============================] - 32s 196ms/step - loss: 0.6064 - acc: 0.8115 - val_loss: 0.9880 - val_acc: 0.6979
Epoch 11/20
162/162 [==============================] - 32s 195ms/step - loss: 0.5609 - acc: 0.8268 - val_loss: 0.8519 - val_acc: 0.7535
Epoch 12/20
162/162 [==============================] - 32s 195ms/step - loss: 0.5207 - acc: 0.8401 - val_loss: 0.8182 - val_acc: 0.7483
Epoch 13/20
162/162 [==============================] - 32s 195ms/step - loss: 0.4556 - acc: 0.8594 - val_loss: 0.6317 - val_acc: 0.7969
Epoch 14/20
162/162 [==============================] - 32s 196ms/step - loss: 0.4426 - acc: 0.8574 - val_loss: 0.5852 - val_acc: 0.7882
Epoch 15/20
162/162 [=============================

Epoch 3/20
162/162 [==============================] - 64s 394ms/step - loss: 0.5013 - acc: 0.9138 - val_loss: 0.3502 - val_acc: 0.9410
Epoch 4/20
162/162 [==============================] - 64s 395ms/step - loss: 0.3895 - acc: 0.9273 - val_loss: 0.2917 - val_acc: 0.9097
Epoch 5/20
162/162 [==============================] - 65s 399ms/step - loss: 0.3043 - acc: 0.9439 - val_loss: 0.1814 - val_acc: 0.9653
Epoch 6/20
162/162 [==============================] - 64s 396ms/step - loss: 0.2527 - acc: 0.9589 - val_loss: 0.1534 - val_acc: 0.9670
Epoch 7/20
162/162 [==============================] - 64s 395ms/step - loss: 0.2098 - acc: 0.9614 - val_loss: 0.1402 - val_acc: 0.9670
Epoch 8/20
162/162 [==============================] - 64s 396ms/step - loss: 0.1833 - acc: 0.9688 - val_loss: 0.1382 - val_acc: 0.9670
Epoch 9/20
162/162 [==============================] - 66s 410ms/step - loss: 0.1620 - acc: 0.9705 - val_loss: 0.0910 - val_acc: 0.9826
Epoch 10/20
162/162 [==============================] - 

Epoch 14/20
162/162 [==============================] - 35s 218ms/step - loss: 0.1683 - acc: 0.9828 - val_loss: 0.1047 - val_acc: 0.9948
Epoch 15/20
162/162 [==============================] - 35s 218ms/step - loss: 0.1462 - acc: 0.9884 - val_loss: 0.1257 - val_acc: 0.9861
Epoch 16/20
162/162 [==============================] - 35s 219ms/step - loss: 0.1345 - acc: 0.9878 - val_loss: 0.0987 - val_acc: 0.9913
Epoch 17/20
162/162 [==============================] - 35s 218ms/step - loss: 0.1218 - acc: 0.9886 - val_loss: 0.0764 - val_acc: 0.9983
Epoch 18/20
162/162 [==============================] - 36s 220ms/step - loss: 0.1022 - acc: 0.9915 - val_loss: 0.0927 - val_acc: 0.9861
Epoch 19/20
162/162 [==============================] - 35s 218ms/step - loss: 0.0942 - acc: 0.9931 - val_loss: 0.0731 - val_acc: 0.9913
Epoch 20/20
5184/5184 [==============================] - 12s 2ms/step
Train [0.07119307790043546, 0.9884259259259259]
1440/1440 [==============================] - 3s 2ms/step
Test [0.0